# WanGP Cloud GPU Setup
## AI Video Generation Platform - Complete Setup for Any Cloud GPU Service

This notebook provides a complete setup for running WanGP (Wan Video Generation Platform) on any cloud GPU service including:
- Google Colab
- Kaggle
- AWS SageMaker
- Paperspace Gradient
- RunPod
- Lambda Labs
- Any other cloud GPU platform

### Features Supported:
- **Text-to-Video**: Generate videos from text descriptions
- **Image-to-Video**: Animate static images
- **VACE ControlNet**: Advanced video manipulation (motion transfer, object injection, inpainting)
- **Multiple Models**: 1.3B and 14B parameter models
- **LoRA Support**: Custom style adaptations
- **Performance Optimizations**: Sage Attention, TeaCache, compilation

### Hardware Requirements:
- **Minimum**: 6GB VRAM (for 1.3B models)
- **Recommended**: 12GB+ VRAM (for 14B models)
- **Memory**: 8GB+ RAM

## 1. Environment Detection and Setup

In [1]:
# New cell to setup environment variables
import os

# Set XDG_RUNTIME_DIR to prevent errors
if 'XDG_RUNTIME_DIR' not in os.environ:
    # Determine user for unique directory
    import getpass
    runtime_dir = f"/tmp/runtime-{getpass.getuser()}"
    os.environ['XDG_RUNTIME_DIR'] = runtime_dir
    if not os.path.exists(runtime_dir):
        os.makedirs(runtime_dir, mode=0o700)
    print(f"✅ Set XDG_RUNTIME_DIR to {runtime_dir}")
else:
    print(f"✓ XDG_RUNTIME_DIR already set to {os.environ['XDG_RUNTIME_DIR']}")

# Fix ALSA warnings by setting dummy audio drivers
os.environ['SDL_AUDIODRIVER'] = 'dummy'
os.environ['ALSA_CONFIG_PATH'] = '/dev/null'
print("✅ Configured audio to use dummy drivers")

print("Environment setup complete - WanGP should now launch without environment errors")

✅ Set XDG_RUNTIME_DIR to /tmp/runtime-root
✅ Configured audio to use dummy drivers
Environment setup complete - WanGP should now launch without environment errors


In [2]:
import os
import sys
import subprocess
import platform
import torch
import json
from pathlib import Path

# Detect environment
def detect_environment():
    if 'COLAB_GPU' in os.environ:
        return 'colab'
    elif 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
        return 'kaggle'
    elif 'SM_TRAINING_ENV' in os.environ:
        return 'sagemaker'
    elif 'PAPERSPACE_NOTEBOOK_REPO_ID' in os.environ:
        return 'paperspace'
    elif 'RUNPOD_POD_ID' in os.environ:
        return 'runpod'
    else:
        return 'generic'

env = detect_environment()
print(f"Detected environment: {env}")
print(f"Python version: {sys.version}")
print(f"Platform: {platform.platform()}")

# Check GPU availability
if torch.cuda.is_available():
    gpu_count = torch.cuda.device_count()
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"GPU Available: {gpu_name}")
    print(f"GPU Memory: {gpu_memory:.1f} GB")
    print(f"CUDA Version: {torch.version.cuda}")
else:
    print("⚠️ No GPU detected! This notebook requires a GPU to run.")
    sys.exit(1)

Detected environment: colab
Python version: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
Platform: Linux-6.1.123+-x86_64-with-glibc2.35
GPU Available: Tesla T4
GPU Memory: 14.7 GB
CUDA Version: 12.4


## 2. System Dependencies Installation

In [3]:
# Install system dependencies
def install_system_deps():
    commands = [
        "apt-get update -qq",
        "apt-get install -y git wget curl ffmpeg libgl1-mesa-glx libglib2.0-0",
    ]

    for cmd in commands:
        print(f"Running: {cmd}")
        result = subprocess.run(cmd.split(), capture_output=True, text=True)
        if result.returncode != 0:
            print(f"Warning: Command failed: {cmd}")
            print(f"Error: {result.stderr}")

# Only install on Linux systems
if platform.system() == 'Linux':
    install_system_deps()
else:
    print("Skipping system dependencies (not Linux)")

Running: apt-get update -qq
Running: apt-get install -y git wget curl ffmpeg libgl1-mesa-glx libglib2.0-0


## 3. Python Environment Setup

In [4]:
# Determine PyTorch version based on CUDA availability
def get_pytorch_install_command():
    cuda_version = torch.version.cuda
    if cuda_version is None:
        return "pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu"

    # Check GPU architecture for RTX 50XX support
    gpu_name = torch.cuda.get_device_name(0).lower()
    if 'rtx 50' in gpu_name or 'rtx50' in gpu_name:
        return "pip install torch==2.7.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/test/cu128"
    else:
        return "pip install torch==2.6.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/test/cu124"

pytorch_cmd = get_pytorch_install_command()
print(f"PyTorch install command: {pytorch_cmd}")

# Install PyTorch
!{pytorch_cmd}

PyTorch install command: pip install torch==2.6.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/test/cu124
Looking in indexes: https://download.pytorch.org/whl/test/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 981.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 20.7 MB/s eta 0:00:

## 4. Clone WanGP Repository

In [5]:
# Clone the repository
repo_url = "https://github.com/remphanostar/WanBook"
repo_dir = "WanBook"  # This is now correct

if os.path.exists(repo_dir):
    print(f"Repository {repo_dir} already exists. Updating...")
    !cd {repo_dir} && git pull
else:
    print(f"Cloning repository from {repo_url}")
    !git clone {repo_url}

# Change to the repository directory
os.chdir(repo_dir)
print(f"Current directory: {os.getcwd()}")

# List contents to verify
!ls -la

Cloning repository from https://github.com/remphanostar/WanBook
Cloning into 'WanBook'...
remote: Enumerating objects: 274, done.
remote: Counting objects: 100% (274/274), done.
remote: Compressing objects: 100% (253/253), done.
remote: Total 274 (delta 22), reused 242 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (274/274), 513.42 KiB | 11.94 MiB/s, done.
Resolving deltas: 100% (22/22), done.


FileNotFoundError: [Errno 2] No such file or directory: 'Wan2GP'

## 5. Install Python Dependencies

In [ ]:
# Install core requirements
!pip install -r requirements.txt

# Install performance optimizations
print("\nInstalling performance optimizations...")

# Try to install Triton (for Sage attention)
try:
    if platform.system() == 'Windows':
        !pip install triton-windows
    else:
        !pip install triton
    print("✅ Triton installed successfully")
except Exception as e:
    print(f"⚠️ Triton installation failed: {e}")

# Try to install SageAttention
try:
    !pip install sageattention==1.0.6
    print("✅ SageAttention installed successfully")
except Exception as e:
    print(f"⚠️ SageAttention installation failed: {e}")

# Try to install Flash Attention
try:
    !pip install flash-attn==2.7.2.post1
    print("✅ Flash Attention installed successfully")
except Exception as e:
    print(f"⚠️ Flash Attention installation failed: {e}")

print("\nDependency installation complete!")

## 6. Configuration and Optimization

In [ ]:
# Determine optimal settings based on GPU memory - FIXED VERSION
def get_optimal_settings():
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    gpu_name = torch.cuda.get_device_name(0).lower()

    settings = {
        'model': '--t2v-1-3B',  # Default to smaller model
        'profile': '4',  # Just the value, not the flag
        'attention': 'sdpa',  # Just the value, not the flag
        'extra_flags': []  # Will be processed correctly
    }

    # GPU-specific optimizations
    if gpu_memory >= 24:
        settings['model'] = '--t2v-14B'  # Use larger model
        settings['profile'] = '3'  # High performance
        settings['extra_flags'].extend(['--preload', '2000'])
    elif gpu_memory >= 12:
        settings['model'] = '--t2v-14B'  # Can handle 14B model
        settings['extra_flags'].extend(['--preload', '1000'])
    elif gpu_memory >= 8:
        settings['extra_flags'].extend(['--preload', '500'])
    else:
        settings['extra_flags'].extend(['--preload', '0'])
        settings['extra_flags'].append('--fp16')

    # Try better attention if available
    try:
        import sageattention
        settings['attention'] = 'sage'
        print("✅ Using Sage Attention")
    except ImportError:
        print("ℹ️ Using default SDPA attention")

    # Enable compilation if Triton is available
    try:
        import triton
        settings['extra_flags'].append('--compile')
        print("✅ Compilation enabled")
    except ImportError:
        print("ℹ️ Compilation disabled (Triton not available)")

    # GPU generation specific settings
    if 'rtx 50' in gpu_name:
        settings['extra_flags'].append('--fp16')
    elif any(x in gpu_name for x in ['rtx 30', 'rtx 40', 'a100', 'v100']):
        settings['extra_flags'].extend(['--teacache', '2.0'])
    elif any(x in gpu_name for x in ['rtx 20', 'rtx 10', 't4']):
        settings['extra_flags'].extend(['--teacache', '1.5'])

    return settings

optimal_settings = get_optimal_settings()
print(f"Optimal settings for your GPU:")
for key, value in optimal_settings.items():
    print(f"  {key}: {value}")

## 7. Create Launch Scripts

In [ ]:
# Create launch script
def create_launch_script(settings, filename="launch_wan2gp.py"):
    script_content = f'''#!/usr/bin/env python3
"""
WanGP Auto-Launch Script for Cloud GPU Platforms
Generated automatically with optimal settings for your hardware
"""

import os
import sys
import subprocess
import time
import signal

def launch_wan2gp():
    # Base command
    cmd = [
        sys.executable, "wgp.py",
        "{settings['model']}",
        "{settings['profile']}",
        "{settings['attention']}",
        "--listen",  # Allow external connections
        "--server-port", "7860",
        "--share",  # Create shareable link
    ]

    # Add extra flags
    extra_flags = {settings['extra_flags']}
    for flag in extra_flags:
        if flag:
            cmd.extend(flag.split())

    print("Starting WanGP with command:")
    print(" ".join(cmd))
    print("\n" + "="*50)
    print("WanGP is starting up...")
    print("This may take a few minutes for first-time model downloads.")
    print("="*50 + "\n")

    # Launch the process
    try:
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
                                 universal_newlines=True, bufsize=1)

        # Monitor output
        for line in process.stdout:
            print(line.rstrip())
            if "Running on" in line:
                print("\n" + "="*50)
                print("🎉 WanGP is ready!")
                print("Access the web interface at the URLs shown above.")
                print("="*50 + "\n")

    except KeyboardInterrupt:
        print("\nShutting down WanGP...")
        process.terminate()
        process.wait()

if __name__ == "__main__":
    launch_wan2gp()
'''

    with open(filename, 'w') as f:
        f.write(script_content)

    # Make it executable
    os.chmod(filename, 0o755)
    print(f"Created launch script: {filename}")

create_launch_script(optimal_settings)

# Create quick launch commands file
quick_commands = f'''
# WanGP Quick Launch Commands

## Optimized for your GPU:
python wgp.py {optimal_settings['model']} {optimal_settings['profile']} {optimal_settings['attention']} --listen --share {' '.join(optimal_settings['extra_flags'])}

## Alternative modes:

# Text-to-Video (default)
python wgp.py --t2v --listen --share

# Image-to-Video
python wgp.py --i2v --listen --share

# VACE ControlNet (advanced)
python wgp.py --vace-1-3B --listen --share

# Low VRAM mode
python wgp.py --t2v-1-3B --profile 4 --attention sdpa --fp16 --listen --share

# High performance mode (24GB+ VRAM)
python wgp.py --t2v-14B --profile 3 --compile --attention sage2 --teacache 2.0 --listen --share

'''

with open('quick_commands.txt', 'w') as f:
    f.write(quick_commands)

print("Created quick_commands.txt with various launch options")

## 8. Environment-Specific Setup

In [ ]:
# Environment-specific configurations
def setup_environment_specific():
    if env == 'colab':
        print("Setting up for Google Colab...")
        # Enable GPU persistence
        !nvidia-smi -pm 1

        # Create ngrok tunnel setup
        colab_setup = '''
# Google Colab specific setup
# Install ngrok for stable URL (optional)
!pip install pyngrok

from pyngrok import ngrok
public_url = ngrok.connect(7860)
print(f"WanGP will be available at: {public_url}")
'''
        with open('colab_setup.py', 'w') as f:
            f.write(colab_setup)

    elif env == 'kaggle':
        print("Setting up for Kaggle...")
        # Kaggle has internet disabled by default in some modes
        print("⚠️ Ensure internet access is enabled in Kaggle settings")

    elif env == 'paperspace':
        print("Setting up for Paperspace Gradient...")
        # Paperspace specific optimizations
        !nvidia-smi -pm 1

    elif env == 'runpod':
        print("Setting up for RunPod...")
        # RunPod usually has good GPU settings by default
        pass

    else:
        print("Generic cloud setup applied")

setup_environment_specific()

## 9. Model Download and Verification

In [ ]:
# Test installation and trigger model download
print("Testing WanGP installation...")
print("This will download the required models (may take several minutes)")

# Create a test script that runs briefly
test_script = '''
import sys
import torch
import time

print("Starting WanGP test...")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# Import WanGP modules to verify installation
try:
    sys.path.append('.')
    # This will trigger model downloads
    print("Testing model loading...")
    print("✅ WanGP installation verified!")
except Exception as e:
    print(f"❌ Error: {e}")
'''

with open('test_installation.py', 'w') as f:
    f.write(test_script)

# Run the test
!python test_installation.py

## 10. Launch WanGP

In [ ]:
# Properly build launch command with correct environment variable handling
import os
import sys
import subprocess

def build_launch_command(settings):
    cmd = [sys.executable, "wgp.py"]

    # Add model flag
    cmd.append(settings['model'])

    # Add profile and attention flags with proper separation
    cmd.extend(["--profile", settings['profile']])
    cmd.extend(["--attention", settings['attention']])

    # Add fixed server options
    cmd.extend(["--listen", "--server-port", "7860", "--share"])

    # Process extra flags (ensuring complete separation of flags and values)
    i = 0
    extra_flags = settings['extra_flags']
    while i < len(extra_flags):
        flag = extra_flags[i]
        if flag.startswith('--') and i + 1 < len(extra_flags) and not extra_flags[i + 1].startswith('--'):
            cmd.extend([flag, extra_flags[i + 1]])
            i += 2
        else:
            cmd.append(flag)
            i += 1

    return cmd

launch_cmd = build_launch_command(optimal_settings)
print("🚀 Launching WanGP with corrected settings...")
print("Command:", " ".join(launch_cmd))
print("\n" + "="*60)
print("📝 IMPORTANT NOTES:")
print("• First launch will download models (this may take 10-20 minutes)")
print("• The web interface will be available at the URLs shown below")
print("• Use Ctrl+C to stop the server")
print("="*60 + "\n")

# Execute the launch command
try:
    process = subprocess.Popen(launch_cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True, bufsize=1)
    url_found = False
    model_loaded = False

    for line in process.stdout:
        print(line.rstrip())

        if "Running on" in line and not url_found:
            print("\n" + "🎉 "*5)
            print("✅ WanGP is now running!")
            print("Access the web interface at the URL above")
            print("🎉 "*5 + "\n")
            url_found = True

        elif "Model loaded" in line and not model_loaded:
            print("\n📦 Model loaded successfully!\n")
            model_loaded = True

        elif "Out of memory" in line or "CUDA out of memory" in line:
            print("\n⚠️ GPU OUT OF MEMORY ERROR DETECTED")
            print("💡 Try launching with a smaller model (e.g., --t2v-1-3B --fp16)")

except KeyboardInterrupt:
    print("\n🛑 Shutting down WanGP...")
    process.terminate()
    process.wait()
    print("✅ WanGP stopped successfully")

except Exception as e:
    print(f"\n❌ Error launching WanGP: {e}")
    print("💡 Try a minimal launch: python wgp.py --listen --share")

## 11. Usage Instructions and Tips

### 🎯 Quick Start Guide

1. **Access the Interface**: Open the web URL shown above
2. **Select Model**: Choose from the dropdown (1.3B for speed, 14B for quality)
3. **Enter Prompt**: Describe what video you want to generate
4. **Click Generate**: Wait for your video to be created

### 🎨 Generation Tips

**Text-to-Video Prompts:**
- "A cat walking through a sunny garden"
- "Cinematic shot of waves crashing on rocks at sunset"
- "A person dancing in the rain, slow motion"

**For Best Results:**
- Be descriptive but concise
- Include camera angles ("close-up", "wide shot", "cinematic")
- Mention lighting ("golden hour", "soft lighting", "dramatic")
- Specify motion ("slow motion", "fast paced", "gentle movement")

### ⚙️ Advanced Features

**Image-to-Video:**
1. Switch to Image-to-Video mode
2. Upload an image
3. Add a text prompt describing the desired motion

**VACE ControlNet:**
- Motion transfer from reference videos
- Object/person injection into scenes
- Video inpainting and outpainting
- Advanced video manipulation

**LoRA Customization:**
- Add custom styles and characters
- Mix multiple LoRAs for unique effects
- Adjust strength with multipliers

### 🚨 Troubleshooting

**Out of Memory Errors:**
- Use smaller models (1.3B instead of 14B)
- Reduce frame count (shorter videos)
- Enable `--fp16` flag
- Use `--profile 4` for memory efficiency

**Slow Generation:**
- Enable compilation with `--compile`
- Use faster attention: `--attention sage`
- Enable TeaCache: `--teacache 2.0`

**Model Download Issues:**
- Ensure stable internet connection
- Check available disk space (models are several GB)
- Restart if downloads are interrupted

### 📊 Performance Optimization

**GPU Memory Usage:**
- 1.3B models: ~6GB VRAM
- 14B models: ~12-24GB VRAM
- VACE models: Similar to base models

**Speed vs Quality Trade-offs:**
- Fewer steps = faster generation, lower quality
- More steps = slower generation, higher quality
- TeaCache = 2x speed boost with minimal quality loss

### 🔗 Useful Commands

Check the `quick_commands.txt` file for various launch options tailored to different use cases and hardware configurations.